In [11]:
# this is just to know how much time will it take to run this entire ipython notebook 
from datetime import datetime
# globalstart = datetime.now()
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns
sns.set_style('whitegrid')
import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

In [4]:
df=pd.read_csv('order_data.csv')


In [5]:
df.tail(1000)

,customer_id,order_id,restaurant_id,ordered_time,pickedup_time,delivered_time,discount_flag,customer_geohash,gmv_total,items,post_status
949457,11126178,279768286,180708,2024-05-20 20:09:08.000,NaN,NaN,0,4nxfejk,0.0,[106176870],Failed
949458,22907246,217309343,250802,2024-05-20 22:47:19.000,2024-05-20 22:53:51.340,2024-05-20 23:10:18.037,0,4nxfvfu,268.0,[105652765],Completed
949459,13370819,207480179,110156,2024-05-20 17:51:43.000,2024-05-20 18:10:57.843,2024-05-20 18:29:04.803,0,4nxfvgw,291.0,[105415145],Completed
949460,11211087,250950698,193347,2024-05-20 19:42:56.000,2024-05-20 20:13:22.204,2024-05-20 20:29:13.214,0,4nxfgf5,381.0,"[107087869, 100163433, 104945560]",Completed
949461,10871052,282454399,124110,2024-05-20 15:09:39.000,2024-05-20 15:39:25.735,2024-05-20 15:45:54.741,0,4nxcg93,129.0,[100584218],Completed
949462,24561499,234269216,342340,2024-05-20 14:31:48.000,2024-05-20 14:43:33.506,2024-05-20 15:03:03.866,0,4nxftsg,104.0,"[103967554, 106922413]",Completed
949463,10505927,281666430,257059,2024-05-20 12:18:52.000,NaN,NaN,0,4nxfujv,0.0,[100836599],Failed
949464,18326295,206429672,186539,2024-05-20 15:41:33.000,2024-05-20 16:01:44.718,2024-05-20 16:14:46.569,0,4nxg7t2,699.0,"[102454406, 100378767, 106287403, 107962066]",Completed
949465,14345912,213814489,174305,2024-05-20 20:41:50.000,2024-05-20 21:08:07.635,2024-05-20 21:14:14.328,0,4nxg73s,548.0,"[103492957, 101826920]",Completed
949466,22508307,257985072,218175,2024-05-20 19:09:45.000,2024-05-20 19:23:55.468,2024-05-20 19:37:21.989,0,4nxfeyz,263.0,[106708524],Completed


In [6]:
df.shape

(950457, 11)

In [7]:
df['items']=df['items'].str.replace('[', '')
df['items']=df['items'].str.replace(']', '')
df['items']=df['items'].str.replace(',', ' ')

df_train=df.iloc[:int(df.shape[0]*0.80)]
df_test=df.iloc[int(df.shape[0]*0.80):]


In [9]:

df_train=pd.concat([pd.Series(row['customer_id'], row['items'].split())              
                    for _, row in df_train.iterrows()]).reset_index()

df_train.columns=['item','user']
df_train=df_train.sort_values(by='user')
df_train.to_csv('train.csv',index=False)

In [22]:
print("Training data ")
print("-"*50)
print("Total no of orders :",df_train.shape[0])
print("Total No of unique users   :"+str(len(np.unique(df_train.user))))
print("Total No of unique items  :"+str(len(np.unique(df_train.item))))

Training data 
--------------------------------------------------
('Total no of orders :', 1546906)
Total No of unique users   :154117
Total No of unique items  :147528


In [24]:

df_test=pd.concat([pd.Series(row['customer_id'], row['items'].split())              
                    for _, row in df_test.iterrows()]).reset_index()

df_test.columns=['item','user']
df_test=df_test.sort_values(by='user')
df_test.to_csv('test.csv',index=False)

In [25]:
print("Test data ")
print("-"*50)
print("Total no of orders :",df_train.shape[0])
print("Total No of unique users   :"+str(len(np.unique(df_test.user))))
print("Total No of unique items  :"+str(len(np.unique(df_test.item))))

Test data 
--------------------------------------------------
('Total no of orders :', 1546906)
Total No of unique users   :78226
Total No of unique items  :84087


In [112]:


df_full=pd.concat([pd.Series(row['customer_id'], row['items'].split())              
                    for _, row in df.iterrows()]).reset_index()

df_full.columns=['item','user']
df_full=df_full.sort_values(by='user')
df_full.to_csv('full.csv',index=False)

In [111]:
#coldstart

In [18]:
import numpy as np
total_users = len(np.unique(df.customer_id))
users_train = len(np.unique(df_train.customer_id))
new_users = total_users - users_train

print('\nTotal number of Users  :', total_users)
print('\nNumber of Users in Train data :', users_train)
print("\nNo of Users that didn't appear in train data: {}({} %) \n ".format(new_users,
                                                                        np.round((new_users/total_users)*100, 2)))

('\nTotal number of Users  :', 52911)
('\nNumber of Users in Train data :', 45682)

No of Users that didn't appear in train data: 7229(0 %) 
 


In [29]:
#reading item dict

df2=pd.read_csv('items_metadata.csv')

df2.head()
print df2.shape
df2.head()

(168636, 4)


,item_id,dish_family,category,cuisine
0,102927323,pav,snacks - evening,north indian
1,102250682,soft drinks,beverage - branded cola/cold,others
2,103846009,breakfast,snacks- others,north indian
3,105415872,dips,sides,others
4,100610198,combos,combos,continental
